In [1]:
import os
import openai
from dotenv import load_dotenv

# 환경 변수를 로드합니다.
load_dotenv()

# API 키를 환경 변수에서 가져옵니다.
API_KEY = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = API_KEY 
openai.api_key = API_KEY

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/practice/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./refer.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. 
            Answer questions using only the following context. 
            If you don't know the answer just say you don't know, don't make it up:
            \n\n
            {context}",
            """
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)
result = chain.invoke("화상을 입으면 어떻게 하죠?")
print(result)

Created a chunk of size 617, which is longer than the specified 600
Created a chunk of size 699, which is longer than the specified 600
Created a chunk of size 2225, which is longer than the specified 600


content='화상을 입으면 화상 부위를 흐르는 찬물 속에 넣어 적어도 10분 동안 담가야 합니다. 상처 부위는 깨끗하고 가능하면 멸균 처리된 보푸라기가 없는 거즈로 덮어야 합니다. 물집은 터뜨리지 말고, 화상 부위에 딱 붙어 있는 물질들은 떼어내지 맙시다. 로션을 바르거나 연고, 기름 같은 것도 바르지 맙시다. 환자를 빨리 병원으로 옮겨야 합니다.'


In [6]:
import nltk
nltk.download('punkt',download_dir='C:\\Users\\rlaal\\AppData\\Roaming\\nltk_data\\tokenizers\\')
nltk.download('punkt_tab',download_dir='C:\\Users\\rlaal\\AppData\\Roaming\\nltk_data\\tokenizers\\')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rlaal\AppData\Roaming\nltk_data\tokenizers\..
[nltk_data]     .
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rlaal\AppData\Roaming\nltk_data\tokenizers\..
[nltk_data]     .
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [ ]:
https://www.safekorea.go.kr/idsiSFK/neo/sfk/cs/contents/prevent/SDIJK14433.html?cd1=33&cd2=999&pagecd=SDIJK144.33&menuSeq=128